In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

import pandas as pd
import geopandas as gpd
from siuba import *

# Finding Transfer Points from Conveyal Paths

* Initial exercise with test data, need Conveyal improvements to scale
* See Slack [discussion](https://cal-itp.slack.com/archives/C02H6JUSS9L/p1701456644851979)

In [2]:
df = pd.read_csv('./Conveyal_Paths_Test.csv')
df.index.rename('rider_trip_id', inplace=True)
df = df.dropna() #  remove same o/d
df >> head(10)

,origin,destination,routes,boardStops,alightStops,rideTimes,accessTime,egressTime,transferTime,waitTimes,totalTime,nIterations
rider_trip_id,,,,,,,,,,,,
1,Archives,Curtis Park,533,7015,7019,9.0,1.1,17.6,0.0,1.9,29.6,1200
2,Archives,Winco,533|013,7034|1096,7043|3089,20.0|21.0,0.3,1.4,2.5,1.7|4.5,51.4,410
3,Archives,Winco,533|023,7034|1096,7043|1464,20.0|22.0,0.3,6.2,2.5,1.7|4.5,57.2,20
4,Archives,Winco,507|026|082,7015|316|1666,7055|1551|3089,16.0|12.0|15.0,1.1,1.4,4.7,1.9|2.7|1.5,56.4,390
5,Archives,Winco,533|023,7034|1096,7043|1464,20.0|21.0,0.3,6.2,2.5,1.7|4.5,56.2,40
6,Archives,Winco,533|084,7034|9801,7049|1074,32.0|10.0,0.3,3.4,1.0,1.7|12.0,60.4,340
7,Archives,Middle School,533|067,7015|2592,7022|2594,16.0|1.0,1.1,8.5,15.9,1.9|4.1,48.5,150
8,Archives,Middle School,533|067,7015|2592,7022|2594,16.0|2.0,1.1,8.5,15.9,1.9|5.1,50.5,450
9,Archives,Middle School,533,7015,7022,16.0,1.1,32.1,0.0,1.9,51.1,600


In [3]:
list(range(3))

[0, 1, 2]

In [4]:
df.columns

Index(['origin', 'destination', 'routes', 'boardStops', 'alightStops',
       'rideTimes', 'accessTime', 'egressTime', 'transferTime', 'waitTimes',
       'totalTime', 'nIterations'],
      dtype='object')

In [5]:
array_cols = ['routes', 'boardStops', 'alightStops',
       'rideTimes', 'waitTimes']

In [6]:
for col in array_cols:
    df.loc[:,col] = df[col].map(lambda x: x.split('|'))

In [7]:
df['xfer_event'] = df.boardStops.map(lambda x: list(range(len(x))))
## add information on which xfer the event represents, with 0 being the rider trip origin (not an xfer)

In [8]:
## not quite, need to drop first stop!
xfer_events = df.explode(array_cols + ['xfer_event'])

In [9]:
xfer_events

,origin,destination,routes,boardStops,alightStops,rideTimes,accessTime,egressTime,transferTime,waitTimes,totalTime,nIterations,xfer_event
rider_trip_id,,,,,,,,,,,,,
1,Archives,Curtis Park,533,7015,7019,9.0,1.1,17.6,0.0,1.9,29.6,1200,0
2,Archives,Winco,533,7034,7043,20.0,0.3,1.4,2.5,1.7,51.4,410,0
2,Archives,Winco,013,1096,3089,21.0,0.3,1.4,2.5,4.5,51.4,410,1
3,Archives,Winco,533,7034,7043,20.0,0.3,6.2,2.5,1.7,57.2,20,0
3,Archives,Winco,023,1096,1464,22.0,0.3,6.2,2.5,4.5,57.2,20,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Airport,Sunrise Mall,001,9801,2807,33.0,0.2,7.8,3.7,4.0,96.8,190,2
218,Airport,Sunrise Mall,db09a072-c976-498b-8824-becced337484,641945e4-f72f-4a09-9110-35d862f24266,b3ee0b92-804a-46b7-932b-e924dcb3ca7c,14.6,0.4,3.7,9.1,1.6,95.7,250,0
218,Airport,Sunrise Mall,Shuttle,SAC,RSV,30.0,0.4,3.7,9.1,3.7,95.7,250,1


In [34]:
xfer_points = (xfer_events >> filter(_.xfer_event > 0)
               >> count(_.boardStops)
               >> arrange (-_.n)
               >> rename(transfer_count = _.n)
              )

In [35]:
xfer_points >> head(5)

,boardStops,transfer_count
45,7069,18
41,7051,13
51,9801,13
17,290,12
20,319,12


## Test with SacRT Feed...

In [36]:
from shared_utils import gtfs_utils_v2
from shared_utils.geography_utils import CA_NAD83Albers
import datetime as dt

In [37]:
analysis_date = dt.date(2023, 11, 22)

In [14]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)
sac_rt = feeds >> filter(_.name.str.contains('Sac'))

In [15]:
sac_rt

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,gtfs_dataset_name,type,name,regional_feed_type
188,bf92aec6721070108bd66c9a9e1ba55a,2023-11-22,75deb87d32ff002508f0b6b0104ddadb,America/Los_Angeles,aHR0cHM6Ly9pcG9ydGFsLnNhY3J0LmNvbS9HVEZTL1NSVE...,cb3074eb8b423dfc5acfeeb0de95eb82,Sacramento Schedule,schedule,Sacramento Schedule,None


In [16]:
stops = gtfs_utils_v2.get_stops(selected_date=analysis_date, operator_feeds=sac_rt.feed_key.to_list())

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_geom'
  sqlalchemy.util.warn(


In [38]:
stops = stops >> select(_.stop_id, _.stop_name,
                       _.geometry)

In [44]:
xfers = stops >> inner_join(_, xfer_points, on = {'stop_id': 'boardStops'})

In [45]:
xfers = xfers.to_crs(CA_NAD83Albers)

In [46]:
xfers >> arrange(-_.transfer_count) >> head(3)

,stop_id,stop_name,geometry,boardStops,transfer_count
32,7069,Mather Field/Mills Station (WB),POINT (-114123.679 63943.273),7069,18
19,9801,WATT AVE & I-80 LRT (SB),POINT (-120265.932 70772.846),9801,13
28,7051,29th Street Station (EB),POINT (-127986.144 61904.823),7051,13


In [47]:
xfers.geometry = xfers.buffer(150)

In [48]:
shapes = gtfs_utils_v2.get_shapes(selected_date=analysis_date, operator_feeds=sac_rt.feed_key.to_list())

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  sqlalchemy.util.warn(


In [59]:
m = shapes.explore(tiles = 'CartoDB positron', color = '#75816B',
                  style_kwds = {'opacity': 0.3})

In [60]:
xfers.explore(column = 'transfer_count', cmap='PuRd', tiles = 'CartoDB positron',
              style_kwds = {'fillOpacity': 0.8},
              m = m)